In [1]:
# OMDB_insert_id = ObjectId('575db02205f6a320145f47d0')


import omdb

import warnings
warnings.filterwarnings('ignore')

import json
import numpy as np

from joblib import Parallel, delayed  
import multiprocessing
num_cores = multiprocessing.cpu_count()

import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.dc_marvel
db.collection_names(include_system_collections=False)


[u'box_office_mojo']

In [2]:
mongo = db.omdb_critics

In [3]:
def getOMDBdata(title):
    if title == 'Fantastic Four(2005)':
        data = omdb.get(title='Fantastic Four', year=2005, fullplot=True, tomatoes=True)
    
    elif title == 'Daredevil':
        data = omdb.get(title='Daredevil', year=2003, fullplot=True, tomatoes=True)
        
    elif title == 'Fantastic Four(2015)':
        data = omdb.get(title='Fantastic Four', year=2015, fullplot=True, tomatoes=True)
    
    elif title == "Marvel's The Avengers":
        data = omdb.get(title='The Avengers', fullplot=True, tomatoes=True)
    
    else:
        data = omdb.get(title=title, fullplot=True, tomatoes=True)
    
    collect = {}
    collect['awards'] = data['awards']
    collect['imdb_id'] = data['imdb_id']
    collect['imdb_rating'] = data['imdb_rating'] 
    collect['metascore'] = data['metascore']
    collect['plot'] = data['plot']
    collect['poster'] = data['poster']
    collect['release_year'] = data['year']
    collect['release_date'] = data['released']
    collect['RT_consensus'] = data['tomato_consensus']
    collect['RT_perc'] = data['tomato_meter']
    collect['RT_rating'] = data['tomato_rating']
    collect['RT_audience_perc'] = data['tomato_user_meter']
    collect['RT_audience_rating'] = data['tomato_user_rating']
    
    return collect
    

In [4]:
Marvel =  ["Howard The Duck", "Blade",
           "X-Men","Blade II",
           "Spider-Man","Daredevil",
           "X-men 2","Hulk",
           "The Punisher","Spider-Man 2",
           "Blade: Trinity", "Elektra",
           "Fantastic Four(2005)","X-Men: The Last Stand",
           "Ghost Rider","Spider-Man 3",
           "Fantastic Four: Rise of the Silver Surfer","Iron Man",
           "The Incredible Hulk","Punisher: War Zone",
           "X-Men Origins: Wolverine","Iron Man 2",
           "Thor","X-Men: First Class",
           "Captain America: The First Avenger",
           "Ghost Rider: Spirit of Vengeance",
           "Marvel's The Avengers","The Amazing Spider-Man",
           "Iron Man 3","The Wolverine",
           "Thor 2: The Dark World","Captain America: The Winter Soldier",
           "The Amazing Spider-Man 2","X-Men: Days of Future Past",
           "Guardians of the Galaxy","Big Hero 6",
           "Avengers: Age of Ultron", "Ant-Man",
           "Fantastic Four(2015)", "Deadpool",
           "Captain America: Civil War"]


Dc = ["Superman","Superman II",
      "Superman III", "Supergirl",
      "Superman IV: The Quest for Peace","Batman",
      "Batman Returns","Batman: Mask of the Phantasm",
      "Batman Forever", "Batman Robin", 
      "Steel", "Catwoman", 
      "Constantine", "Batman Begins",
      "V for Vendetta", "Superman Returns", 
      "The Dark Knight", "Watchmen",
      "The Losers","Jonah Hex","Red",
      "Green Lantern", "The Dark Knight Rises",
      "Man of Steel","Red 2",
      "Batman v Superman: Dawn of Justice"]

In [5]:
%%capture


DC = Parallel(n_jobs=num_cores)(delayed(getOMDBdata)(title) for title in Dc)
MARVEL = Parallel(n_jobs=num_cores)(delayed(getOMDBdata)(title) for title in Marvel)

M = {}
D = {}
for k in range(len(MARVEL)):
    M[Marvel[k]]= MARVEL[k]
    
for k in range(len(DC)):
    D[Dc[k]]= DC[k]

    
Data = {}
Data['Marvel'] = M
Data['DC'] = D

del M
del D
del Marvel
del MARVEL
del DC
del Dc

In [6]:
OMDB_insert_id = mongo.insert_one(Data).inserted_id
db.collection_names(include_system_collections=False)

[u'omdb_critics', u'box_office_mojo']

In [7]:

client.close

<bound method MongoClient.close of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

In [8]:
OMDB_insert_id 

ObjectId('575f5d9c05f6a326adc2381f')